## Inverse Reinforcement Learning for Financial Cliff Walking


This notebook contains implementations of three IRL algorithms for the Financial Cliff Walking (FCW) problem:

Max Causal Entropy IRL

IRL from Failure (IRLF)

T-REX

In [28]:
import matplotlib.pyplot as plt 
%matplotlib inline

import numpy as np
import time

import warnings
warnings.filterwarnings('ignore')

In [30]:
# Global variables
# N - World height
# T - World width
WORLD_HEIGHT = 4
WORLD_WIDTH = 12

# Probability for exploration - epsilon
EPSILON = 0.1
# Step size
ALPHA = 0.001
# Gamma - discount factor - for Q-Learning, Sarsa and Expected Sarsa
GAMMA = 0.9

# Actions - ACTION_UP is a+ (adding a deposit), ACTION_DOWN is a-(redeeming a deposit) and 
# ACTION_ZERO is a0 (leaving the account as it is).
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_ZERO = 2
ACTIONS = [ACTION_UP, ACTION_DOWN, ACTION_ZERO]

# Initial and Goal states
START = [1,0]
GOAL = [0, WORLD_WIDTH-1]

### Functions determining the step

In [33]:
# Step function that describes how the next state is obtained from the current state and the action 
# taken. The function returns the next state and the reward obtained.
def step(state, action):
    i, j = state

    if state[0] == 0 and (state[1] > 0): #  and state[1] < WORLD_WIDTH - 2):
        # remain in the bankruptcy state
        next_state =  [0, min(j + 1, WORLD_WIDTH - 1)]
        reward = 0 
        return next_state, reward
    
    # if at the final time, next state is the same, and reward is zero
    if state[1] == WORLD_WIDTH - 1:
        next_state = [i,state[1]]
        reward = 0
        return next_state, reward
    
    if action == ACTION_UP:
        next_state = [min(i + 1, WORLD_HEIGHT-1), min(j + 1, WORLD_WIDTH - 1)]
    elif action == ACTION_DOWN:
        next_state = [max(i - 1, 0), min(j + 1, WORLD_WIDTH - 1)]
    elif action == ACTION_ZERO:
        next_state = [i, min(j + 1, WORLD_WIDTH - 1)]
    else:
        assert False
    
    # The reward is -1 for actions ACTION_UP and ACTION_DOWN. This is done to keep transactions to a minimum.
    reward = -1
    
    # ACTION_ZERO gets a zero reward since we want to minimize the number of transactions
    if action == ACTION_ZERO:
        reward = 0
    
    # Exceptions are 
    # i) If bankruptcy happens before WORLD_WIDTH time steps
    # ii) No deposit at initial state
    # iii) Redemption at initial state!
    # iv) Any action carried out from a bankrupt state
    if ((action == ACTION_DOWN and i == 1 and 1 <= j < 10) or (
        action == ACTION_ZERO and state == START) or (
        action == ACTION_DOWN and state == START )) or (
        i == 0 and 1 <= j <= 10):    
            reward = -100
        
    # Next exception is when we get to the final time step.
    if state[0] != 0 and (next_state[1] == WORLD_WIDTH - 1): 
        # override a random action by a deterministic action=ACTION_DOWN
        if (next_state[0] == 0): # Action resulted in ending with zero balance in final time step
            reward = 10
        else:
            reward = -10   
        
    return next_state, reward

# Choose an action based on epsilon greedy algorithm
def choose_action(state, q_value, eps=EPSILON):
    if np.random.binomial(1, eps) == 1:
        action = np.random.choice(ACTIONS)
    else:
        values_ = q_value[state[0], state[1], :]
        action = np.random.choice([action_ for action_, value_ in enumerate(values_) 
                                 if value_ == np.max(values_)])
    # From bankrupt state there is no meaningful action, so we will assign 'Z' by convention.
    if state[0] == 0 and state[1] > 0:
        action = ACTION_ZERO
    return action

In [35]:
def pos2idx(pos):
    """
    input:
      column-major 2d position
    returns:
      1d index
    """
    return np.ravel_multi_index((pos[0], pos[1]), (WORLD_HEIGHT, WORLD_WIDTH))

def idx2pos(idx):
        
    """
    input:
      1d idx
    returns:
      2d column-major position
    """
    
    pos = [np.unravel_index(idx, (WORLD_HEIGHT, WORLD_WIDTH))[0],
           np.unravel_index(idx, (WORLD_HEIGHT, WORLD_WIDTH))[1]]
    return pos 


def get_transition_mat():
    
    """
    get transition dynamics of the gridworld

    return:
      P_a         NxNxN_ACTIONS transition probabilities matrix - 
                    P_a[s0, s1, a] is the transition prob of 
                    landing at state s1 when taking action 
                    a at state s0
                    
    Probabilities are ones for transitions to states produced by function step(state,action)
    """
    n_states = WORLD_HEIGHT * WORLD_WIDTH
    n_actions = len(ACTIONS)
    
    P_a = np.zeros((n_states, n_states, n_actions))
    
    for si in range(1, n_states): 
        for a in ACTIONS:            
            next_state, reward = step(idx2pos(si),a)
            sj = pos2idx(next_state)                         
            P_a[int(si), int(sj), int(a)] = 1

    return P_a

def get_true_rewards():
    """
    get the reward matrix of shape n_states x n_actions
    """
    n_states = WORLD_HEIGHT * WORLD_WIDTH
    n_actions = len(ACTIONS)
    
    gt_rewards = np.zeros((n_states, n_actions))
    for si in range(n_states):
        for a in ACTIONS:            
            next_state, reward = step(idx2pos(si),a)                        
            gt_rewards[int(si), int(a)] = reward
    
    return gt_rewards
    
def get_rewards_on_rectangle(rewards, action): # matrix of size n_state x n_action
                            
    # convert rewards into rectangular form WORLD_HEIGHT x WORLD_WIDTH
    rewards_rectangle = np.zeros((WORLD_HEIGHT, WORLD_WIDTH))
    n_states = rewards.shape[0]
    
    for idx in range(n_states):
        height, width = idx2pos(idx)
        rewards_rectangle[height,width] = rewards[idx,action]
    
    return rewards_rectangle

def get_values_on_rectangle(values): # here values are given by the vector of size n_states
    
    values_rectangle = np.zeros((WORLD_HEIGHT, WORLD_WIDTH))
    n_states = len(values)
    
    for idx in range(n_states):
        height, width = idx2pos(idx)
        values_rectangle[height,width] = values[idx]
    
    return values_rectangle
    
def value_iteration(P_a, rewards, gamma, error=0.01, deterministic=False):
    
    
    """
    static value iteration function
  
    inputs:
    P_a         N x N x N_ACTIONS transition probabilities matrix - 
                              P_a[s0, s1, a] is the transition prob of 
                              landing at state s1 when taking action 
                              a at state s0
    rewards     N x N_ACTIONS matrix - rewards for all the state-action combinations
    gamma       float - RL discount
    error       float - threshold for a stop
    deterministic   bool - to return deterministic policy or stochastic policy
  
    returns:
    values    Nx1 matrix - estimated values
    policy    Nx1 (NxN_ACTIONS if non-det) matrix - policy
    """
    n_states, _, n_actions = np.shape(P_a)

    values = np.zeros([n_states])

    # estimate values
    while True:
        values_tmp = values.copy()

        for s in range(n_states):
            v_s = []
            values[s] = max([sum([P_a[s, s1, a]*(rewards[s,a] + gamma*values_tmp[s1]) for s1 in range(n_states)]) 
                             for a in ACTIONS])    
        
        if max([abs(values[s] - values_tmp[s]) for s in range(n_states)]) < error:
            break


    if deterministic:
        # generate deterministic policy
        policy = np.zeros([n_states])
        for s in range(n_states):
            policy[s] = np.argmax([sum([P_a[s, s1, a]*(rewards[s,a]+gamma*values[s1]) 
                                  for s1 in range(n_states)]) 
                                  for a in ACTIONS])

        return values, policy
    else:
        # generate stochastic policy
        policy = np.zeros([n_states, n_actions])
        for s in range(n_states):
            v_s = np.array([sum([P_a[s, s1, a]*(rewards[s,a] + gamma*values[s1]) for s1 in range(n_states)]) 
                            for a in ACTIONS])
            policy[s,:] = np.transpose(v_s/np.sum(v_s))
        return values, policy
    
    
def soft_Q_iteration(rewards, gamma, beta, learn_rate_Q, n_states, n_actions, 
                     trajs, error=0.01, max_iter = 500):
    """
    Performs soft Q-iteration based on observed samples 
    Returns: the soft-Q function, soft-V function, and soft-Q policy
    
    """

    T = trajs.shape[1] 
    n_traj = trajs.shape[0]
    
    discounts = np.ones(T)
    for t in range(1,T):
        discounts[t] *= gamma
        
    # initialize
    soft_Q_vals =  np.zeros((n_states, n_actions))
    soft_Q_policy = np.zeros((n_states, n_actions))
    
    iter_idx = 0
    
    # soft-Q iteration
    while True:
        soft_Q_vals_tmp = soft_Q_vals.copy()
        
        for episode in range(n_traj):
            for t in range(T):
                state = trajs[episode,t,0]
                action = trajs[episode,t,1]
                next_state = trajs[episode,t,2]
                reward = rewards[state, action]
                
                soft_V_vals_next = (1.0/beta) * np.log(np.sum(np.exp(beta*soft_Q_vals[next_state,:])))
                soft_Q_vals[state, action] += learn_rate_Q*(reward + gamma * soft_V_vals_next 
                                                            - soft_Q_vals[state, action])
        
        iter_idx += 1
        if np.max(soft_Q_vals - soft_Q_vals_tmp) < error or iter_idx >= max_iter:
            break

    # soft V-function        
    soft_V_vals = (1.0/beta) * np.log(np.sum(np.exp(beta*soft_Q_vals),axis=1))
    
    # stochastic policy
    policy = np.exp(beta*(soft_Q_vals - np.tile(soft_V_vals[:, np.newaxis],(1,n_actions) )))
            
    return soft_Q_vals, soft_V_vals, policy


def soft_Q_iteration_mb(P_a, rewards, gamma, beta, n_states, n_actions, error=0.01):
    """
    Soft Q-iteration based on a transition model (model-based soft Q-iteration)
    """
    
    Q_values = np.zeros((n_states, n_actions))

    # estimate Q-values
    while True:
        Q_values_tmp = Q_values.copy()

        for s in range(n_states):
            for a in range(n_actions):   
                Q_values[s,a] = np.sum(P_a[s,:,a]*(rewards[s,a]
                                        + (gamma/beta)*np.log(np.sum(np.exp(beta*Q_values_tmp[:,:]),axis=1))))
        if np.max(np.abs(Q_values - Q_values_tmp)) < error:
            break

    # soft V-function        
    soft_V_vals = (1.0/beta) * np.log(np.sum(np.exp(beta*Q_values),axis=1))
    
    # stochastic policy
    policy = np.exp(beta*(Q_values - np.tile(soft_V_vals[:, np.newaxis],(1,n_actions) )))
            
    return Q_values, soft_V_vals, policy            
        
def compute_state_visition_freq(P_a, gamma, T, policy, deterministic=False):
    
    """compute the expected states visition frequency p(s| theta, T) 
    using dynamic programming

    inputs:
    P_a     N x N x N_ACTIONS matrix - transition dynamics
    gamma   float - discount factor
    trajs   list of list of Steps - collected from expert
    policy  N x 1 vector (or NxN_ACTIONS if deterministic=False) - policy
    T       number of time steps
  
    returns:
    svf       N x 1 vector - state visitation frequencies
    savf      N x N_ACTION - state-action visitation frequencies
    """
    n_states, _, n_actions = np.shape(P_a)
    
    # mu[s, t] is the prob of visiting state s at time t
    mu = np.zeros([n_states, T]) 
    
    # as all trajectories in our case start at [0,0], initialize the time-dependent state density at t = 0
    mu[pos2idx(START),0] = 1

    # compute mu[s,t] for other times 
    for s in range(n_states):
        for t in range(T-1):
            if deterministic:
                mu[s, t+1] = sum([mu[pre_s, t]*P_a[pre_s, s, int(policy[pre_s])] 
                                  for pre_s in range(n_states)])
            else:
                mu[s, t+1] = sum([sum([mu[pre_s, t]*P_a[pre_s, s, a1]*policy[pre_s, a1] 
                                       for a1 in ACTIONS]) for pre_s in range(n_states)])

    
    # array of discount factors
    discounts = np.ones(T)
    for i in range(1,T):
        discounts[i] = gamma * discounts[i-1]
    
    # state visitation frequencies
    svf = np.sum(mu * discounts, 1)
        
    # state-action visitation frequencies
    savf = np.tile(svf[:,np.newaxis],(1,n_actions)) * policy
    
    return svf, savf


def maxent_irl(feat_map, P_a, gamma, beta, lr, lr_Q, trajs, n_iters):
    
    """
    Maximum Entropy Inverse Reinforcement Learning (Maxent IRL)

    inputs:
        feat_map    NxD matrix - the features for each state
        P_a         NxNxN_ACTIONS matrix - P_a[s0, s1, a] is the transition prob of 
                                           landing at state s1 when taking action 
                                           a at state s0
        gamma       float - RL discount factor
        beta        regularization parameter for soft Q-learning
        lr          learning rate
        lr_Q        learning rate for soft Q-learning
        trajs       a list of demonstrations
        
        n_iters     int - number of optimization steps

    returns
        rewards     Nx1 vector - recoverred state rewards
    """
    n_states, _, n_actions = np.shape(P_a)

    # init parameters
    
    # theta is a matrix of parameters of shape n_states x n_actions, for actions = 0,1,2
    theta = np.random.uniform(low=-0.1,high=0.1, size=(feature_matrix.shape[1],n_actions))    

    # calc feature expectations
    feat_exp = np.zeros((feat_map.shape[1],n_actions))
    
    T = trajs.shape[1]
    n_traj = trajs.shape[0]
    
    discounts = np.ones(T)
    for t in range(1,T):
        discounts[t] *= gamma
    
    for episode in range(n_traj):
        for t in range(T):
            for a in ACTIONS:
                state = trajs[episode,t,0]
                action = trajs[episode,t,1]
                feat_exp[state, action] += discounts[t]
    
    feat_exp = feat_exp/n_traj 
    
    # training
    for iteration in range(n_iters):
  
        if iteration % (n_iters/20) == 0:
            print('iteration: {}/{}'.format(iteration, n_iters))
    
        # compute reward function as a matrix of shape n_states x n_actions, for actions = 0,1,2
        rewards = np.dot(feat_map, theta)
        
        # try computing policy using model-based soft Q-iteration
        _, _, policy = soft_Q_iteration_mb(P_a, rewards, gamma, beta, 
                                           n_states, n_actions, error=0.01)
    
        # compute state-action visition frequences
        _, savf = compute_state_visition_freq(P_a, gamma, T, policy, deterministic=False)
            
        # compute gradients
        grad = feat_exp - feat_map.T.dot(savf) # now gradient has shape n_states x n_actions

        # update params
        theta += lr * grad

    rewards = np.dot(feat_map, theta)
    return rewards 

# use this function from irl-imitation for displaying results
def heatmap2d(hm_mat, title='', block=True, fig_num=1, text=True):
    
    """
    Display heatmap
    input:
        hm_mat:   mxn 2d np array
    """
    print ('map shape: {}, data type: {}'.format(hm_mat.shape, hm_mat.dtype))

    if block:
        plt.figure(fig_num)
        plt.clf()
  
    plt.imshow(hm_mat, interpolation='nearest',origin="lower")
    plt.title(title)
    plt.colorbar()
  
    if text:
        for y in range(hm_mat.shape[0]):
            for x in range(hm_mat.shape[1]):
                plt.text(x, y, '%.1f' % hm_mat[y, x],
                     horizontalalignment='center',
                     verticalalignment='center',
                     )

    if block:
        plt.ion()
        print ('press enter to continue')
        plt.show()
        raw_input()

### IRLF

In [38]:
def irl_from_failure(feat_map, P_a, gamma, 
                     beta,
                     trajs_success, 
                     trajs_failure,
                     lr,
                     lr_Q,
                     alpha_lam,
                     lam,
                     lam_min, 
                     n_iters):
    
    """
    Maximum Entropy Inverse Reinforcement Learning (Maxent IRL)

    inputs:
        feat_map    NxD matrix - the features for each state
        P_a         NxNxN_ACTIONS matrix - P_a[s0, s1, a] is the transition prob of 
                                           landing at state s1 when taking action 
                                           a at state s0
        gamma               float - RL discount factor
        trajs_success       successful demonstrations
        trajs_failure       failed demonstrations
        lr          float - learning rate
        n_iters     int - number of optimization steps

    returns
        rewards     Nx1 vector - recoverred state rewards
    """
    n_states, _, n_actions = np.shape(P_a)

    # init parameters
    
    # theta_s and theta_f are sets of parameters for successful and failed trajectories
    
    # theta_s is a matrix of parameters of shape n_states x n_actions, for actions = 0,1,2
    theta_s = np.random.uniform(low=-0.1,high=0.1, size=(feature_matrix.shape[1],n_actions))    
    theta_f = np.zeros((feature_matrix.shape[1], n_actions))
    
    # calc feature expectations
    feat_exp_s = np.zeros((feat_map.shape[1],n_actions))
    feat_exp_f = np.zeros((feat_map.shape[1],n_actions))
    
    T_s = trajs_success.shape[1]
    n_traj_s = trajs_success.shape[0]
    T_f = trajs_failure.shape[1] 
    n_traj_f = trajs_failure.shape[0]
    
    # discount factors
    T = T_s
    discounts = np.ones(T)
    for t in range(1,T):
        discounts[t] *= gamma
    
    # empirical feature expectations for successful trajectories
    for episode in range(n_traj_s):
        for t in range(T):
            for a in ACTIONS:
                state = trajs_success[episode,t,0]
                action = trajs_success[episode,t,1]
                feat_exp_s[state, action] += discounts[t]
    
    feat_exp_s = feat_exp_s/n_traj_s 
    
    # empirical feature expectations for failed trajectories
    for episode in range(n_traj_f):
        for t in range(T):
            for a in ACTIONS:
                state = trajs_failure[episode,t,0]
                action = trajs_failure[episode,t,1]
                feat_exp_f[state, action] += discounts[t]
    
    feat_exp_f = feat_exp_f/n_traj_f 
    
    # training
    for iteration in range(n_iters):
  
        if iteration % (n_iters/20) == 0:
            print('iteration: {}/{}'.format(iteration, n_iters))
    
        # compute reward function as a matrix of shape n_states x n_actions, for actions = 0,1,2
        rewards = np.dot(feat_map, theta_s + theta_f)

        # compute policy (using only successful trajectories?)
        
        # try computing policy using model-based soft Q-iteration
        _, _, policy = soft_Q_iteration_mb(P_a, rewards, gamma, beta, 
                                           n_states, n_actions, error=0.01)
    
        # compute state-action visition frequency
        _, savf_s = compute_state_visition_freq(P_a, gamma, T_s, policy, deterministic=False)
            
        # as both successful and failed trajectories start with the same initial state and 
        # have the same length, the state-action visitation frequency for failed trajectories is the same
        savf_f = savf_s.copy()
            
        # compute gradients
        grad_theta_s = feat_exp_s - feat_map.T.dot(savf_s) # now gradient has shape n_states x n_actions

        # update params
        theta_s += lr * grad_theta_s
        
        # recompute weights theta_f
        theta_f = (savf_f - feat_exp_f)/lam
        
        if lam > lam_min:
            lam = alpha_lam * lam

    return rewards, policy, theta_s, theta_f